# Information Extraction - Football Players

In [2]:
import nltk
import re
from statistics import mode

In [3]:
inputfile='football_players.txt' #Location of the file
buf=open(inputfile, encoding="UTF-8")
list_of_doc=buf.read().split('\n')

21


# Task 1
Write a function that takes each document and performs:
1) sentence segmentation 2) tokenization 3) part-of-speech tagging

Please keep in mind that the expected output is a list within a list as shown below.


In [5]:
def ie_preprocess(document):
    pos_sentences=[]
    sentences=nltk.sent_tokenize(document)
    for sentence in sentences:
        tokenized_sentence=nltk.word_tokenize(sentence)
        pos_sentences.append(nltk.pos_tag(tokenized_sentence))
    return pos_sentences

Run the following code to check your result for the first document (Ronaldo).

In [286]:
first_doc=list_of_doc[0]
pos_sent=ie_preprocess(first_doc)
pos_sent

[[('Cristiano', 'NNP'),
  ('Ronaldo', 'NNP'),
  ('dos', 'NN'),
  ('Santos', 'NNP'),
  ('Aveiro', 'NNP'),
  (',', ','),
  ('ComM', 'NNP'),
  (',', ','),
  ('GOIH', 'NNP'),
  ('(', '('),
  ('born', 'VBN'),
  ('5', 'CD'),
  ('February', 'NNP'),
  ('1985', 'CD'),
  (')', ')'),
  ('is', 'VBZ'),
  ('a', 'DT'),
  ('Portuguese', 'JJ'),
  ('professional', 'JJ'),
  ('footballer', 'NN'),
  ('who', 'WP'),
  ('plays', 'VBZ'),
  ('for', 'IN'),
  ('Spanish', 'JJ'),
  ('club', 'NN'),
  ('Real', 'NNP'),
  ('Madrid', 'NNP'),
  ('and', 'CC'),
  ('the', 'DT'),
  ('Portugal', 'NNP'),
  ('national', 'JJ'),
  ('team', 'NN'),
  ('.', '.')],
 [('He', 'PRP'),
  ('is', 'VBZ'),
  ('a', 'DT'),
  ('forward', 'NN'),
  ('and', 'CC'),
  ('serves', 'NNS'),
  ('as', 'IN'),
  ('captain', 'NN'),
  ('for', 'IN'),
  ('Portugal', 'NNP'),
  ('.', '.')],
 [('In', 'IN'),
  ('2008', 'CD'),
  (',', ','),
  ('he', 'PRP'),
  ('won', 'VBD'),
  ('his', 'PRP$'),
  ('first', 'JJ'),
  ('Ballon', 'NNP'),
  ("d'Or", 'NN'),
  ('and', 'CC')

Expected output
 [...[('He', 'PRP'),
  ('is', 'VBZ'),
  ('a', 'DT'),
  ('forward', 'NN'),
  ('and', 'CC'),
  ('serves', 'NNS'),
  ('as', 'IN'),
  ('captain', 'NN'),
  ('for', 'IN'),
  ('Portugal', 'NNP'),
  ('.', '.')], ...]

# Task 2
Write a function that will take the list of tokens with POS tags for each sentence and returns the named entities (NE). 

Hint: Use binary=True while calling NE chunk function

In [288]:
def named_entity_finding(pos_sent):
    tree=nltk.ne_chunk(pos_sent, binary=True)
    
    #print (tree)
    
    named_entities = []
    for subtree in tree.subtrees():
        if subtree.label() == 'NE':
            entity = ""
            for leaf in subtree.leaves():
                entity = entity + leaf[0] + " "
            named_entities.append(entity.strip())
    return named_entities

first_doc=list_of_doc[0]
pos_sents=ie_preprocess(first_doc)
named_entity_finding(pos_sents[0])

['Cristiano Ronaldo',
 'Santos Aveiro',
 'ComM',
 'GOIH',
 'Portuguese',
 'Spanish',
 'Real Madrid',
 'Portugal']

Expected output ['Cristiano Ronaldo',
 'Santos Aveiro',
 'ComM',
 'GOIH',
 'Portuguese',
 'Portuguese',
 'Spanish',
 'Real Madrid',
 'Portugal']

# Task 3

Now use the named_entity_finding() function to extract all NEs for each document.

Hint: pos_sents holds the list of lists of tokens with POS tags

In [11]:
def NE_flat_list_fn(pos_sents): 
    NE=[]
    for pos_sent in pos_sents:
        NE.append(named_entity_finding(pos_sent))
    NE_flat_list=[ne_item for ne_list in NE for ne_item in ne_list]
    return NE_flat_list

# Task 4

Write functions to extract the name of the player, country of origin and date of birth as well as the following relations: team(s) of the player and position(s) of the player.

Hint: Use the re.compile() function to create the extraction patterns

Reference: https://docs.python.org/3/howto/regex.html

In [454]:
import re

def name_of_the_player(doc):
    name=""
    pos_sents=ie_preprocess(doc)
    grammar = r"PERSON: {<NNP>+<FW|NN|\``>*<NNP>*<\''>*<NNP>+}"
    chunker = nltk.RegexpParser(grammar)
    tree = chunker.parse(pos_sents[0])
    for subtree in tree.subtrees():
        if subtree.label() == 'PERSON':
            phrase = ""
            for leaf in subtree.leaves():
                phrase = phrase + leaf[0] + " "
            name=phrase.strip()
            break
    return name

name_of_the_player(list_of_doc[8])

'Wayne Mark Rooney'

In [291]:
def country_of_origin(doc):
    country=""
    pattern = re.compile(r'the\s([\S]+)\snational team')
    matches=pattern.finditer(doc)
    for match in matches:
        country=match[1]
        break
    return country

country_of_origin(list_of_doc[8])

'England'

In [292]:
def date_of_birth(doc):
    date=""
    pattern = re.compile(r'born\b\s(\d+\s[\S]+\s\d+)')
    matches=pattern.finditer(doc)
    for match in matches:
        date=match[1]
    return date

date_of_birth(list_of_doc[8])

'24 October 1985'

In [459]:
def team_of_the_player(doc):
    team=set([])
    team.add(country_of_origin(doc)+" national team")
    
    pattern1 = re.compile(r"played\sfor\s(.*?)national\steam")
    pattern2 = re.compile(r"club\s(FC\s\S+)")
    pattern3 = re.compile(r"play\sfor\s(.\w*)")
    pattern4 = re.compile(r"career\sat\s(.*?)\s")
    pattern5 = re.compile(r"signed\sby\s(\S+\s)(\S+)")
    pattern6 = re.compile(r"transferred\sto\s(\S+)\s(\S+)")
    pattern7 = re.compile(r"clubs\s((?:\S+\s){5})")
    
    for match in pattern1.finditer(doc):
        [team.add(club) for club in match[1].split(",")]
    for match in pattern2.finditer(doc):
        [team.add(club) for club in match[1].split(",")]
    for match in pattern3.finditer(doc):
        [team.add(club) for club in match[1].split(",")]
    for match in pattern4.finditer(doc):
        [team.add(club) for club in match[1].split(",")]
    for match in pattern5.finditer(doc):
        if(str(match[2].split(",")[0])!="where"):
            team.add(str(match[1].split(",")[0])+" "+str(match[2].split(",")[0]))
        else:
            team.add(str(match[1].split(",")[0]))
    for match in pattern6.finditer(doc):
        if(str(match[2].split(",")[0])!="before"):
            team.add(str(match[1].split(",")[0])+" "+str(match[2].split(",")[0]))
        else:
            team.add(str(match[1].split(",")[0]))
    for match in pattern7.finditer(doc):
        [team.add(club) for club in match[1].split(",")]
        
    return list(team)

team_of_the_player(list_of_doc[4])

['FC Barcelona', 'Brazil national team']

In [294]:
def position_of_the_player(doc):
    positions = ["forward", "captian", "attacking midfielder", "striker", "winger", "central midfielder", "defensive tackle", "defensive end"]
    position = set([])
    for pos in positions:
        pattern = re.compile(r'\b({0})\b'.format(pos), flags=re.IGNORECASE)
        if bool(pattern.search(doc)):
            position.add(pos)
    return list(position)

position_of_the_player(list_of_doc[8])

['forward']

Execute the below command to check your fuction


In [455]:
date_of_birth(list_of_doc[2])

'24 June 1987'

Expected output '5 February 1992'

# Task 5

Write a function using the outputs from the previous functions to generate JSON-LD output as follows.

Reference: https://json-ld.org/primer/latest/

{ "@id": "http://my-soccer-ontology.com/footballer/name_of_the_player",

    "name": "",
    "born": "",
    "country": "",
    "position": [
        { "@id": "http://my-soccer-ontology.com/position",
            "type": ""
        }
     ]   
     "team": [
        { "@id": "http://my-soccer-ontology.com/team",
            "name": ""
        }   
     ]
}


In [498]:
def generate_jsonld(arg):
    ld = { "@id": "http://my-soccer-ontology.com/footballer/"+arg[0],

            "name": arg[0],
            "born": arg[1],
            "country": arg[2],
            "position": [
                { "@id": "http://my-soccer-ontology.com/position/",
                    "type": arg[3]
                }
             ],   
             "team": [
                { "@id": "http://my-soccer-ontology.com/team/",
                    "name": arg[4]
                }   
             ]
        }

    return json.dumps(ld, indent=4, ensure_ascii=False)

def get_values_as_list(doc):
    return [name_of_the_player(doc), date_of_birth(doc), country_of_origin(doc), position_of_the_player(doc), team_of_the_player(doc)]

data=get_values_as_list(list_of_doc[8])
print (generate_jsonld(data))

{
    "@id": "http://my-soccer-ontology.com/footballer/Wayne Mark Rooney",
    "name": "Wayne Mark Rooney",
    "born": "24 October 1985",
    "country": "England",
    "position": [
        {
            "@id": "http://my-soccer-ontology.com/position/",
            "type": [
                "forward"
            ]
        }
    ],
    "team": [
        {
            "@id": "http://my-soccer-ontology.com/team/",
            "name": [
                "England national team"
            ]
        }
    ]
}


# Task 6
Identify one other relation (besides team and player) and write a function to extract this. Also extend the JSON-LD output accordingly.

In [466]:
# finding the no. of awards earned by the player.
# award value is N/A if it is not available

def award_of_the_player(doc):
    awards = set([])
    pattern = re.compile(r'(\S+\s+){0,8}\b[Aa]wards*\b')
    matches=pattern.finditer(doc)
    for match in matches:
        a=re.search(r'\b(win|won)\b', match[0])
        if a is None:
            a=re.search(r'\b(and)\b', match[0])
            b=re.search(r'\b(and was)\b', match[0])
            if (a is None or b is not None):
                a=re.search(r'\b(the)\b', match[0])
        index=a.start()
        awards.add(match[0][index+len(a[0].strip())+1:len(match[0])])
       
    if len(awards)==0:
        return ['N/A']
    else:
        return list(awards)

award_of_the_player(list_of_doc[16])

['N/A']

In [500]:
import json
#Generating Json ld for the updated method (get player award)
def generate_jsonld_updated(arg):
    ld = { "@id": "http://my-soccer-ontology.com/footballer/"+arg[0],
            "name": arg[0],
            "born": arg[1],
            "country": arg[2],
            "position": [
                { "@id": "http://my-soccer-ontology.com/position/",
                    "type": arg[3]
                }
             ],   
             "team": [
                { "@id": "http://my-soccer-ontology.com/team/",
                    "name": arg[4]
                }   
             ],
              "award": [
                { "@id": "http://my-soccer-ontology.com/award/",
                    "name": arg[5]
                }   
             ]
            }

    return json.dumps(ld, indent=4, ensure_ascii=False)

def get_values_as_list_updated(doc):
    return [name_of_the_player(doc), date_of_birth(doc), country_of_origin(doc), position_of_the_player(doc), team_of_the_player(doc), award_of_the_player(doc)]

data=get_values_as_list_updated(list_of_doc[8])
print (generate_jsonld_updated(data))

{
    "@id": "http://my-soccer-ontology.com/footballer/Wayne Mark Rooney",
    "name": "Wayne Mark Rooney",
    "born": "24 October 1985",
    "country": "England",
    "position": [
        {
            "@id": "http://my-soccer-ontology.com/position/",
            "type": [
                "forward"
            ]
        }
    ],
    "team": [
        {
            "@id": "http://my-soccer-ontology.com/team/",
            "name": [
                "England national team"
            ]
        }
    ],
    "award": [
        {
            "@id": "http://my-soccer-ontology.com/award/",
            "name": [
                "the England Player of the Year award",
                "Premier League Goal of the 20 Seasons award",
                "the Premier League Player of the Month award",
                "the Goal of the Season award"
            ]
        }
    ]
}
